In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [2]:
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
final_table_attributes = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'

In [3]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n') 

In [38]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Name": col[1].find_all('a')[-1].contents[0],
                            "MC_USD_Billion": col[2].contents[0].strip('\n')}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    return df

In [39]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0)['Rate'].to_dict()
    df['MC_GBP_Billion'] = [np.round(float(x)*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(float(x)*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(float(x)*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    return df

In [41]:
def load_to_csv(df, output_path):
    df.to_csv(csv_path, index=False)

In [42]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [46]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [52]:
# Extract
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, 'exchange_rate.csv')
log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect(db_name)
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')

query1 = "SELECT * FROM Largest_banks"
run_query(query1, sql_connection)
print('-------------------------------------')
query2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query2, sql_connection)
print('-------------------------------------')
query3 = "SELECT Name from Largest_banks LIMIT 5"
run_query(query3, sql_connection)
print('-------------------------------------')

log_progress('Process Complete.')

sql_connection.close()

SELECT * FROM Largest_banks
                                      Name MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase         432.92          346.34   
1                          Bank of America         231.52          185.22   
2  Industrial and Commercial Bank of China         194.56          155.65   
3               Agricultural Bank of China         160.68          128.54   
4                                HDFC Bank         157.91          126.33   
5                              Wells Fargo         155.87          124.70   
6                        HSBC Holdings PLC         148.90          119.12   
7                           Morgan Stanley         140.83          112.66   
8                  China Construction Bank         139.82          111.86   
9                            Bank of China         136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.9